# Setup

In [ ]:
import os
import sys
import random
sys.path.append("..")
debug = True # If True, will produce plots and/or verbose output to double-check
rerun_existing = False

# Path
print("Current working directory: ", sys.path[0])
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "graph_similarity.py"
%run -i "write_results_percentages.py"
%run -i "analyze_poi_based_results.py"
%run -i "generate_network_plots.py"

%load_ext watermark
%watermark -n -v -m -g -iv

In [ ]:
place_ids = ["copenhagen"]
iterations = 100

prune_quantiles = [0.25, 0.5, 0.75, 1]
subgraph_percentages_runs = [
    [0.6, 0.4],
    [0.25, 0.25, 0.25, 0.25],
    [0.75, 0.25],
    [0.25, 0.75],
    [0.33, 0.33, 0.34],
    [1],
]

prune_measures = ["betweenness", "closeness", "random"]

# Run the program

In [ ]:
for placeid in tqdm(place_ids, desc="Cities", leave=False):
    print(placeid + ": Generating graph and POIs")

    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

    for prune_measure in tqdm(prune_measures, desc="Prune measures", leave=False):
        (GTs, GT_abstracts) = greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)
        (MST, MST_abstract) = mst_routing(G_carall, nnids)

        # Feed the number of edges added for each prune quantile to our subgraph generation
        # to ensure that the same number of edges are added in our own method
        number_of_edges_to_add = list(map(lambda g: g.ecount(), GT_abstracts))

        for subgraph_percentages in tqdm(subgraph_percentages_runs, desc="Cities", leave=False):
            subgraph_percentage_string = '_'.join(map(str, subgraph_percentages))
            file_name = f"{placeid}_{prune_measure}_{subgraph_percentage_string}.csv"
            dir = "csv_data/"
            path = os.path.join(dir, file_name)
            with open(path, 'w') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "sum_of_errors",
                    "continuous_vertices",
                    "bikengrowth_vertices",
                    "continuous_disconnected_pairs",
                    "bikengrowth_disconnected_pairs",
                    "prune_quantile",
                    "continuous_abstract_edges",
                    "bikengrowth_abstract_edges",
                ])

                for i in tqdm(range(iterations), desc="Iterations", leave=False):
                    if prune_measure == "random":
                        (GTs, GT_abstracts) = greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)
                        number_of_edges_to_add = list(map(lambda g: g.ecount(), GT_abstracts))

                    gts, abstract_gts = iterative_greedy_triangulation_routing(
                        G_carall, nnids, subgraph_percentages, number_of_edges_to_add, prune_measure
                    )

                    for prune_quantile, gt, szell_gt, abstract_gt, szell_abstract_gt in zip(prune_quantiles, gts, GTs, abstract_gts, GT_abstracts):
                        error, szell_disconnected_points, our_disconnected_points = sum_of_errors_pois(szell_gt, gt, nnids)
                        writer.writerow([error, gt.vcount(), szell_gt.vcount(), our_disconnected_points, szell_disconnected_points, prune_quantile, abstract_gt.ecount(), szell_abstract_gt.ecount()])

                    results = {"placeid": placeid, "prune_measure": prune_measure, "poi_source": poi_source, "prune_quantiles": prune_quantiles, "GTs": gts, "GT_abstracts": abstract_gts, "MST": MST, "MST_abstract": MST_abstract}
                    write_result_with_percentages(res=results, mode="pickle", placeid=placeid, iteration=str(i), subgraph_percentage=subgraph_percentage_string, poi_source=poi_source, prune_measure=prune_measure, suffix=".pickle")

In [ ]:
string_list_subgraph_percentage = []
for subgraph_percentage in subgraph_percentages_runs:
    string_list_subgraph_percentage.append('_'.join(map(str, subgraph_percentage)))

In [ ]:
# Analyze results
for placeid in tqdm(place_ids, desc="Cities", leave=False):
    analyze_existing_infrastructure([placeid])
    analyze_poi_base_results_iterations(placeid=placeid, subgraphs_percentages=string_list_subgraph_percentage, iterations=iterations)

In [ ]:
#generate_plots
for placeid in tqdm(place_ids, desc="Cities", leave=False):
    existing_network_plot(placeid=placeid, prune_measures=prune_measures)

In [ ]:
for placeid in tqdm(place_ids, desc="Cities", leave=False):
    subgraph_percentages_with_iterations_plot(placeid=placeid, pms=prune_measures, subgraphs_percentages=string_list_subgraph_percentage, iterations=iterations)